In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
### from models.py
from models import *

In [27]:
# load the model
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")
model = Model_Rational_Label.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")

def prediction(text):
    inputs = tokenizer(text, return_tensors="pt")
    prediction_logits, _ = model(input_ids=inputs['input_ids'],attention_mask=inputs['attention_mask'])
    label = prediction_logits.argmax().item()
    # 1 means hate speech, 0 means non-hate speech

    return label

In [29]:
# load the data
df = pd.read_csv("hate.csv")

In [30]:
df

,post_id,label,target,text,prefix,suffix
0,1179088797964763136_twitter,hatespeech,Asian,<user> i am bit confused coz chinese ppl can n...,<user> i am bit confused coz chinese ppl can n...,how this ching chong using it i think he pakis...
1,1179085312976445440_twitter,hatespeech,Caucasian,this bitch in whataburger eating a burger with...,this bitch in whataburger eating a burger with...,bun and holding it palm up i hate white bitches
2,13851720_gab,hatespeech,Jewish,laura loomer raped me while screaming at me in...,laura loomer raped me while screaming at me in...,kike language and said we must exterminate the...
3,24198545_gab,hatespeech,African,and this is why i end up with nigger trainee d...,and this is why i end up with nigger trainee d...,speak properly lack basic knowledge of biology...
4,19679648_gab,hatespeech,African,guilty until proven innocent unless you are a ...,guilty until proven innocent unless you are a,jew a nigger or a lover of both
...,...,...,...,...,...,...
5930,9867117_gab,hatespeech,Islam,well my dear lgbtq brothers and sisters i do n...,well my dear lgbtq brothers and sisters i do n...,as you are putting my life in danger with your...
5931,9875482_gab,hatespeech,African,c mon men charge it for the gays trannies musl...,c mon men charge it for the gays trannies muslim,rapists so that an effeminate half black guy c...
5932,9981407_gab,hatespeech,Jewish,<user> congrats on making the kike hate list w...,<user> congrats on making the kike hate,list whoohoo be careful out there brother
5933,9982674_gab,hatespeech,Arab,more and more leftists are moving closer to ce...,more and more leftists are moving closer to ce...,chaos going importing muslim fighters is now t...


In [31]:
# predict labels for prefix and suffix sentence
df['prefix_label'] = df['prefix'].apply(prediction)
df['suffix_label'] = df['suffix'].apply(prediction)

In [35]:
# chech the distribution of labels
print(df['prefix_label'].value_counts())

print(df['suffix_label'].value_counts())

1    3382
0    2553
Name: prefix_label, dtype: int64
1    4298
0    1637
Name: suffix_label, dtype: int64


In [37]:
print(df[(df['prefix_label']==0) & (df['suffix_label']==0)].shape)

(173, 8)


In [40]:
# a new dataframe to save the non-hate predix or suffix sentences
df_new = df[(df['prefix_label']==0) | (df['suffix_label']==0)]
# remove duplicate sentences
df_new = df_new.drop_duplicates(subset=['prefix', 'suffix'], keep='first')

In [41]:
df_new

,post_id,label,target,text,prefix,suffix,prefix_label,suffix_label
0,1179088797964763136_twitter,hatespeech,Asian,<user> i am bit confused coz chinese ppl can n...,<user> i am bit confused coz chinese ppl can n...,how this ching chong using it i think he pakis...,0,1
2,13851720_gab,hatespeech,Jewish,laura loomer raped me while screaming at me in...,laura loomer raped me while screaming at me in...,kike language and said we must exterminate the...,0,1
3,24198545_gab,hatespeech,African,and this is why i end up with nigger trainee d...,and this is why i end up with nigger trainee d...,speak properly lack basic knowledge of biology...,1,0
4,19679648_gab,hatespeech,African,guilty until proven innocent unless you are a ...,guilty until proven innocent unless you are a,jew a nigger or a lover of both,0,1
6,23844035_gab,hatespeech,Islam,<number> <number> <percent> of brits think mul...,<number> <number> <percent> of brits think mul...,their muzzie master there no great in britain ...,0,1
...,...,...,...,...,...,...,...,...
5927,9844016_gab,hatespeech,African,gas all niggers jews the world over presto mos...,gas all niggers jews the world over,presto most if not all racism ends,1,0
5929,9845733_gab,hatespeech,Jewish,do not for get my followers the jews have decl...,do not for get my followers the jews have decl...,this time and we will not just asked them to l...,1,0
5932,9981407_gab,hatespeech,Jewish,<user> congrats on making the kike hate list w...,<user> congrats on making the kike hate,list whoohoo be careful out there brother,1,0
5933,9982674_gab,hatespeech,Arab,more and more leftists are moving closer to ce...,more and more leftists are moving closer to ce...,chaos going importing muslim fighters is now t...,0,1


In [42]:
# a new dataframe to save the non-hate predix or suffix sentences
df_prompt = pd.DataFrame(columns=['post_id', 'text', 'part'])

for index, row in df_new.iterrows():
    if row['prefix_label']==0:
        df_prompt = df_prompt.append({'post_id':row['post_id'], 'text':row['prefix'], 'part':'prefix'}, ignore_index=True)
    if row['suffix_label']==0:
        df_prompt = df_prompt.append({'post_id':row['post_id'], 'text':row['suffix'], 'part':'suffix'}, ignore_index=True)


/tmp/ipykernel_22079/2294230684.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_prompt = df_prompt.append({'post_id':row['post_id'], 'text':row['prefix'], 'part':'prefix'}, ignore_index=True)
/tmp/ipykernel_22079/2294230684.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_prompt = df_prompt.append({'post_id':row['post_id'], 'text':row['prefix'], 'part':'prefix'}, ignore_index=True)
/tmp/ipykernel_22079/2294230684.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_prompt = df_prompt.append({'post_id':row['post_id'], 'text':row['suffix'], 'part':'suffix'}, ignore_index=True)
/tmp/ipykernel_22079/2294230684.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

In [44]:
# randomly fill the tokens for the user, number, percent, etc.
import random

def fill_tokens(text):
    # tokens = [['<user>', '<number>', '<percent>', '<time>', '<date>', '<location>']]
    tokens = text.split()
    for token in tokens:
        if token == '<user>':
            tokens[tokens.index(token)] = '@' + str(random.randint(100000, 999999))
        if token == '<number>':
            tokens[tokens.index(token)] = str(random.randint(100000, 999999))
        if token == '<percent>':
            tokens[tokens.index(token)] = str(random.randint(1, 100)) + '%'
        if token == '<time>':
            tokens[tokens.index(token)] = str(random.randint(0, 23)) + ':' + str(random.randint(0, 59))
        if token == '<date>':
            tokens[tokens.index(token)] = str(random.randint(1, 12)) + '/' + str(random.randint(1, 31)) + '/' + str(random.randint(2000, 2020))
        if token == '<location>':
            tokens[tokens.index(token)] = 'location' + str(random.randint(100000, 999999))
    return ' '.join(tokens)

In [45]:
# fill the tokens for the prompt
df_prompt['text'] = df_prompt['text'].apply(fill_tokens)

In [47]:
df_prompt.groupby('part').count()

,post_id,text
part,,
prefix,2550,2550
suffix,1635,1635


In [48]:
# save the prompt
df_prompt.to_csv('prompt.csv', index=False)

In [49]:
df_prompt

,post_id,text,part
0,1179088797964763136_twitter,@769288 i am bit confused coz chinese ppl can ...,prefix
1,13851720_gab,laura loomer raped me while screaming at me in...,prefix
2,24198545_gab,speak properly lack basic knowledge of biology...,suffix
3,19679648_gab,guilty until proven innocent unless you are a,prefix
4,23844035_gab,173750 263867 47% of brits think multicultural...,prefix
...,...,...,...
4180,9844016_gab,presto most if not all racism ends,suffix
4181,9845733_gab,this time and we will not just asked them to l...,suffix
4182,9981407_gab,list whoohoo be careful out there brother,suffix
4183,9982674_gab,more and more leftists are moving closer to ce...,prefix
